In [6]:
! pip3 install paramiko psycopg2
! pip3 install psycopg2-binary

In [ ]:
! /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
! brew install postgresql

==> Checking for `sudo` access (which may request your password)...
Password:
sudo: a password is required
Need sudo access on macOS (e.g. the user aoitsuno needs to be an Administrator)!
==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 4 taps (hashicorp/tap, homebrew/services, homebrew/core and homebrew/cask).
==> New Formulae
anubis              fedify              ktexttemplate       pyp
apache-flink@1      fortran-stdlib      lavat               statesmith
block-goose-cli     ftxui               leetgo              tfcmt
chdig               gama                libpg_query         tsui
cpp-lazy            gamdl               lld@19              twitch-cli
darklua             gdown               llvm@19             veccore
dblab               geesefs             mani                x-cmd
dish                

In [9]:
import psycopg2
import psycopg2.extras
import json

# PostgreSQL接続情報
DB_HOST = "127.0.0.1"
DB_PORT = 5433
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "mypass"

QUERY_DATABASES = """
    SELECT * from courses;
"""

# テーブルとカラム情報を取得するSQLクエリ
QUERY_TABLES = """
    SELECT 
        c.relname AS table_name, 
        COALESCE(d.description, '') AS table_comment
    FROM pg_catalog.pg_class c
    LEFT JOIN pg_catalog.pg_description d ON d.objoid = c.oid
    WHERE c.relkind = 'r';
"""

QUERY_COLUMNS = """
    SELECT 
        c.table_name, 
        c.column_name, 
        c.data_type,
        COALESCE(d.description, '') AS column_comment
    FROM information_schema.columns c
    LEFT JOIN pg_catalog.pg_description d 
        ON d.objoid = (
            SELECT oid FROM pg_catalog.pg_class 
            WHERE relname = c.table_name 
            AND relnamespace = (
                SELECT oid FROM pg_catalog.pg_namespace 
                WHERE nspname = c.table_schema
            )
        )
        AND d.objsubid = c.ordinal_position
    WHERE c.table_schema NOT IN ('pg_catalog', 'information_schema');
"""

def fetch_catalog_data():
    """PostgreSQLのスキーマ情報を取得し、カタログデータを生成"""
    try:
        # PostgreSQLへ直接接続
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            cursor_factory=psycopg2.extras.DictCursor
        )
        cur = conn.cursor()

        # テーブル情報を取得
        cur.execute(QUERY_TABLES)
        tables = {table: {"table": table, "description": comment, "columns": []} for table, comment in cur.fetchall()}

        # カラム情報を取得
        cur.execute(QUERY_COLUMNS)
        for table_name, column_name, data_type, column_comment in cur.fetchall():
            if table_name in tables:
                tables[table_name]["columns"].append({
                    "name": column_name,
                    "type": data_type,
                    "description": column_comment
                })

        # JSON形式に整形
        catalog_data = list(tables.values())
        json_output = json.dumps(catalog_data, indent=4, ensure_ascii=False)
        print(json_output)

        # 接続を閉じる
        cur.close()
        conn.close()

        return json_output

    except Exception as e:
        print(f"Error: {e}")

# カタログデータを取得
fetch_catalog_data()


[
    {
        "table": "departments",
        "description": "学部名",
        "columns": [
            {
                "name": "id",
                "type": "integer",
                "description": "学部ID (主キー)"
            },
            {
                "name": "name",
                "type": "character varying",
                "description": "学部名"
            }
        ]
    },
    {
        "table": "students",
        "description": "学部ID (departments.id への外部キー)",
        "columns": [
            {
                "name": "id",
                "type": "integer",
                "description": "学生ID (主キー)"
            },
            {
                "name": "year",
                "type": "integer",
                "description": "学年 (1～4)"
            },
            {
                "name": "department_id",
                "type": "integer",
                "description": "学部ID (departments.id への外部キー)"
            },
            {
                "name": "name",
            

'[\n    {\n        "table": "departments",\n        "description": "学部名",\n        "columns": [\n            {\n                "name": "id",\n                "type": "integer",\n                "description": "学部ID (主キー)"\n            },\n            {\n                "name": "name",\n                "type": "character varying",\n                "description": "学部名"\n            }\n        ]\n    },\n    {\n        "table": "students",\n        "description": "学部ID (departments.id への外部キー)",\n        "columns": [\n            {\n                "name": "id",\n                "type": "integer",\n                "description": "学生ID (主キー)"\n            },\n            {\n                "name": "year",\n                "type": "integer",\n                "description": "学年 (1～4)"\n            },\n            {\n                "name": "department_id",\n                "type": "integer",\n                "description": "学部ID (departments.id への外部キー)"\n            },\n            {\n      